In [1]:
import pandas as pd
pd.set_option("display.max_columns",250)
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import yaml as yaml
# import openpyxl
import random

sns.set()
# import openmatrix as omx
# import geopandas
# import pyproj
# import re
# import xlrd
from IPython.display import display, HTML

In [2]:
def add_percentage_to_crosstab(ct):
    total_counts = ct['All'][:-1].sum()
    ct['Percent'] = ct.apply(lambda row: round(100 * row['All'] / total_counts, 2), axis=1)
    ct.loc['Percent'] = ct.apply(lambda col: round(100 * col['All'] / total_counts, 2), axis=0)
    ct.loc['Percent', 'Percent'] = pd.NA
    return ct

In [3]:
def make_percentage_bar_plot(variable, title, xlabel, xlabel_rotation):
    grouped_df = obs_df.groupby(variable)["unlinked_weight_scaled"].agg(sum)
    total_weight = grouped_df.sum()
    grouped_df["percent"] = grouped_df/total_weight*100
    plt.figure(figsize=(10,7))
    grouped_df["percent"].plot.bar()
    plt.xlabel(xlabel, fontsize=16)
    plt.xticks(rotation=xlabel_rotation)
    plt.ylabel("Weighted Percent", fontsize=16)
    plt.title(title, fontsize=16)
    plt.show()
    
def make_percentage_bar_plot_cta(variable, title, xlabel, xlabel_rotation):
    grouped_df = obs_df.groupby(variable)["unlinked_weight_scaled"].agg(sum)
    total_weight = grouped_df.sum()
    grouped_df["percent"] = grouped_df/total_weight*100
    plt.figure(figsize=(10,7))
    grouped_df["percent"].plot.bar()
    plt.xlabel(xlabel, fontsize=16)
    plt.xticks(rotation=xlabel_rotation)
    plt.ylabel("Weighted Percent", fontsize=16)
    plt.title(title, fontsize=16)
    plt.show()
    
def make_percentage_bar_plot_metra(variable, title, xlabel, xlabel_rotation):
    grouped_df = metra_df.groupby(variable)["unlinked_weight_scaled"].agg(sum)
    total_weight = grouped_df.sum()
    grouped_df["percent"] = grouped_df/total_weight*100
    plt.figure(figsize=(10,7))
    grouped_df["percent"].plot.bar()
    plt.xlabel(xlabel, fontsize=16)
    plt.xticks(rotation=xlabel_rotation)
    plt.ylabel("Weighted Percent", fontsize=16)
    plt.title(title, fontsize=16)
    plt.show()

In [5]:

with open(r'E:\Met_Council\metc-asim-model\source\survey_data_processing\metc_inputs_phase2.yml') as file:
    settings = yaml.load(file)

obs_path  = os.path.join(settings['obs_dir'], 'Met Council OD 2022.xlsx', 'data')

obs_df = pd.read_xlsx(obs_path)

output_path = r"output"


TypeError: load() missing 1 required positional argument: 'Loader'

In [6]:
print("Total entries in the on board survey:", len(obs_df))
print("Weighted total of entries in the on board survey:", obs_df['Unlinked_Weight-AE'].sum())



Total entries in the on board survey: 30605
Weighted total of entries in the on board survey: 350474.127932107


# Transit Onboard Survey Data Coding

## Recode relevant variables to match between each dataset

In [7]:

obs_df_orig = obs_df.copy()

list(obs_df)


['ID',
 'DATE',
 'ROUTE_SURVEYED_CODE',
 'ROUTE_SURVEYED',
 'HOME_OR_HOTEL_ADDR_CITY',
 'HOME_OR_HOTEL_ADDR_STATE',
 'HOME_OR_HOTEL_ADDR_ZIP',
 'HOME_OR_HOTEL_ADDR_LAT_100M',
 'HOME_OR_HOTEL_ADDR_LON_100M',
 'BOARDING_LOCATION',
 'BOARDING_STOPID',
 'BOARDING_LAT',
 'BOARDING_LON',
 'TRANSFERS_FROM_CODE',
 'TRANSFERS_FROM',
 'TRANSFERS_FROM_FIRST_CODE',
 'TRANSFERS_FROM_FIRST',
 'TRANSFERS_FROM_FIRST_OTHER',
 'TRANSFERS_FROM_SECOND_CODE',
 'TRANSFERS_FROM_SECOND',
 'TRANSFERS_FROM_SECOND_OTHER',
 'TRANSFERS_FROM_THIRD_CODE',
 'TRANSFERS_FROM_THIRD',
 'TRANSFERS_FROM_THIRD_OTHER',
 'TRANSFERS_FROM_FOURTH_CODE',
 'TRANSFERS_FROM_FOURTH',
 'ORIGIN_PLACE_TYPE_CODE',
 'ORIGIN_PLACE_TYPE',
 'ORIGIN_CITY',
 'ORIGIN_STATE',
 'ORIGIN_ZIP',
 'ORIGIN_LAT_100M',
 'ORIGIN_LON_100M',
 'ACCESS_MODE_CODE',
 'ACCESS_MODE',
 'VEHICLE_ACCESS_PARK_YN_PNR_CODE',
 'VEHICLE_ACCESS_PARK_YN_PNR',
 'VEHICLE_ACCESS_PARK_PNR_CODE',
 'VEHICLE_ACCESS_PARK_PNR',
 'VEHICLE_ACCESS_PARK_PNR_OTHER',
 'VEHICLE_ACCESS_DRO

In [8]:
obs_df = obs_df_orig[['ID', 'ROUTE_SURVEYED_CODE', 'ROUTE_SURVEYED', 'ROUTE', 'ROUTE_PROVIDER','ORIGIN_PLACE_TYPE_CODE','ORIGIN_PLACE_TYPE', 
                      'ACCESS_MODE_CODE','ACCESS_MODE', 'EGRESS_MODE_CODE', 'EGRESS_MODE',
                     'DESTIN_PLACE_TYPE_CODE', 'DESTIN_PLACE_TYPE', 
                      'TIME_ON', 'TIME_PERIOD', 'TIME_ON_CODE',
                      'TRIP_PURPOSE_NO_OTHER_TRIP',
                     'TRIP_PURPOSE_BUY_A_MEAL_/_BEVERAGE',
                     'TRIP_PURPOSE_GO_TO_WORK',
                     'TRIP_PURPOSE_VISIT_FRIEND/RELATIVE_OR_ATTEND_RELIGIOUS/SOC_EVENT',
                     'TRIP_PURPOSE_GO_TO_SCHOOL',
                     'TRIP_PURPOSE_GO_SHOPPING',
                     'TRIP_PURPOSE_OTHER_ERRANDS',
                     'TRIP_PURPOSE_OTHER',
                      'COUNT_EMPLOYED_HH_CODE', 'COUNT_EMPLOYED_HH',
                      'COUNT_VH_HH_CODE', 'COUNT_VH_HH',
                      'COUNT_MEMBER_HH_CODE', 'COUNT_MEMBER_HH', 'STATUS_EMPLOYMENT', 'STUDENT_STATUS',
                    'HAS_DRIVE_LICENSE', 'AGE', 'INCOME_CODE', 'INCOME', 
                      'O_TAZ', 'D_TAZ', 'Unlinked_Weight-AE', 'Linked_Weight-AE']].copy()





### Boardings by route


Questions:

In [9]:
obs_route_df = obs_df.groupby(['ROUTE_PROVIDER'])['Unlinked_Weight-AE'].agg(sum).sort_index(level='ROUTE_PROVIDER').to_frame()
obs_route_df['Percent'] = obs_route_df['Unlinked_Weight-AE'] / obs_route_df['Unlinked_Weight-AE'].sum() * 100

# print(route_dict.head())
display(HTML(obs_route_df.to_html()))

,Unlinked_Weight-AE,Percent
ROUTE_PROVIDER,,
MVTA,10386.425806,2.963536
Maple Grove,3509.824504,1.001450
Met Council,11222.951323,3.202220
Metro Transit,297493.573266,84.883177
Plymouth,1829.987180,0.522146
Southwest,4286.000000,1.222915
U of M,21745.365854,6.204557


In [10]:
def code_purpose(value):
    if (value == 2):
        return 'Work'
    elif (value == 1):
        return 'Home'
    elif (value == 3):
        return 'University'
    elif (value == 9): # airport
        return 'Maintenance'
    elif (value == 8): # social
        return 'Discretionary'
    elif (value == 5): # medical
        return 'Maintenance'
    elif (value == 7): # recreation
        return 'Discretionary'
    elif (value == 6): # shop
        return 'Maintenance'
    elif (value == 10): # sporting
         return 'Discretionary'
    elif (value == 'Shopping'):
        return 'Maintenance'
    elif (value == 11): # work related
        return 'Maintenance'
    
    elif (value == 12): # hotel
        return 'Maintenance'
    elif (value == 4):
        return 'K-12 school'
    elif (value == 99):
        return 'Maintenance'





In [11]:
obs_df['ORIG_PURPOSE'] = obs_df['ORIGIN_PLACE_TYPE_CODE'].apply(lambda value: code_purpose(value))

obs_df['DEST_PURPOSE'] = obs_df['DESTIN_PLACE_TYPE_CODE'].apply(lambda value: code_purpose(value))


In [12]:
print(obs_df['ORIG_PURPOSE'].value_counts())
print(obs_df['DEST_PURPOSE'].value_counts())


Home             15295
Work              8175
Discretionary     2856
Maintenance       2053
University        1629
K-12 school        483
Name: ORIG_PURPOSE, dtype: int64
Home             11691
Work              9339
Discretionary     4294
Maintenance       2424
University        2170
K-12 school        573
Name: DEST_PURPOSE, dtype: int64


### Access mode

* 1 - PNR (driving, carshare)
* 2 - KNR (dropped off, taxi/TNC)
* 3 - Walk (includes bike, scooter, skateboard)

Questions:
* Should taxi be TNC?


In [13]:
# Code access/egress before imputation

def code_access_mode(value):
    if (value in  [4, 9, 10, 3, 12, 8]):
        return 'KNR'
    elif (value in [7, 6, 5]):
        return 'PNR'
    return 'Walk'


obs_df['access_mode'] = obs_df['ACCESS_MODE_CODE'].apply(lambda value: code_access_mode(value))
obs_df['egress_mode'] = obs_df['EGRESS_MODE_CODE'].apply(lambda value: code_access_mode(value))


In [14]:
orig_dest_ct = pd.crosstab(obs_df['access_mode'], obs_df['egress_mode'], values=obs_df['Unlinked_Weight-AE'], aggfunc=sum, margins=True)
orig_dest_ct = add_percentage_to_crosstab(orig_dest_ct)
orig_dest_ct.fillna(0, inplace=True)
orig_dest_ct.applymap(lambda x: round(x,2))


egress_mode,KNR,PNR,Walk,All,Percent
access_mode,,,,,
KNR,3965.11,125.69,8574.30,12665.11,3.61
PNR,98.44,302.24,19553.46,19954.14,5.69
Walk,6329.09,18673.86,292851.93,317854.88,90.69
All,10392.64,19101.79,320979.69,350474.13,100.00
Percent,2.97,5.45,91.58,100.00,0.00


### Auto Sufficiency
* 0 for households with no autos
* 1 for households that have 0 < autos < number of workers
* 2 for households that have 0 < autos >= number of workers

Questions:
* Auto suff for Metra was imputed in R based on CMAP data
  * Mainly resulted in auto suff "vehicles greater than or equal to workers"
  * Does that make sense for commuter rail?
* Some missing hh employed or hh veh in CTA

In [15]:
obs_df['auto_suff'] = -1



In [16]:

obs_df.loc[obs_df.COUNT_VH_HH_CODE != '10P', 'hh_veh'] = obs_df.COUNT_VH_HH_CODE
obs_df.loc[obs_df.COUNT_VH_HH_CODE == '10P','hh_veh'] = 10
obs_df['hh_veh'] = pd.to_numeric(obs_df['hh_veh'])

obs_df.loc[pd.isna(obs_df.COUNT_EMPLOYED_HH_CODE), 'hh_employed'] = 0

obs_df['hh_employed'] = pd.to_numeric(obs_df['COUNT_EMPLOYED_HH_CODE'])
obs_df.loc[obs_df.hh_veh < obs_df.hh_employed, 'auto_suff'] = 1
obs_df.loc[obs_df.hh_veh >= obs_df.hh_employed, 'auto_suff'] = 2
obs_df.loc[obs_df.hh_veh == 0, 'auto_suff'] = 0



In [17]:
print(obs_df['auto_suff'].value_counts(normalize = True))
print(obs_df[obs_df['auto_suff'] >=0]['auto_suff'].value_counts(normalize = True))
print(obs_df['hh_veh'].value_counts())
print(obs_df['hh_employed'].value_counts())



 2    0.366509
 0    0.325241
 1    0.258585
-1    0.049665
Name: auto_suff, dtype: float64
2    0.385663
0    0.342238
1    0.272099
Name: auto_suff, dtype: float64
0.0     9954
1.0     8978
2.0     7264
3.0     2016
4.0      591
5.0      184
6.0       54
10.0      23
7.0       13
8.0        5
9.0        3
Name: hh_veh, dtype: int64
2.0     11801
1.0      8619
3.0      4030
0.0      2417
4.0      1485
5.0       437
6.0       142
10.0       75
7.0        46
8.0        22
9.0        11
Name: hh_employed, dtype: int64


In [18]:
# impute autosuff based on overall distribution

# 0    0.342
# 1    0.272
# 2    0.386
random.seed(413)

m = obs_df[obs_df['auto_suff']<0].shape[0]

s = np.random.choice([0, 1, 2],
                     p=[ 0.342, 0.272,0.386], 
                     size = m)
print(s)

obs_df.loc[obs_df['auto_suff'] < 0, 'auto_suff'] = s

[2 0 1 ... 2 0 2]


In [19]:
# # Scale to boardings in oct 2018
# pd.set_option('display.max_rows', 130)

# metcouncil_boardings = ...

# Assume updated weights will have right boarding sum?
# Scale missing purposes


weight_sum = obs_df['Unlinked_Weight-AE'].sum()
linked_weight_sum =  obs_df['Linked_Weight-AE'].sum()
print(weight_sum)
print(linked_weight_sum)
obs_df = obs_df[~obs_df['ORIGIN_PLACE_TYPE_CODE'].isnull()]

weight_sum_2 =  obs_df['Unlinked_Weight-AE'].sum()
linked_weight_sum_2 =  obs_df['Linked_Weight-AE'].sum()

obs_df['unlinked_weight_scaled'] = obs_df['Unlinked_Weight-AE'] * (weight_sum/weight_sum_2)
obs_df['linked_weight_scaled'] = obs_df['Linked_Weight-AE'] * (linked_weight_sum/linked_weight_sum_2)
print('scaled weights')
print(obs_df.unlinked_weight_scaled.sum())
print(obs_df.linked_weight_scaled.sum())


350474.127932107
272362.71444986097
scaled weights
350474.127932107
272362.71444986097


#### Impute tour purposes for non-work tours:

Impute work tours/sub-tours and joint tour purposes where applicable using time of day.

* Impute work tours from maintenance/discretionary tours 
* Impute work sub-tours where neither end is home
* Impute joint tours from maintenance/discretionary tours that remain

Metra is only first half of the day, so have to do it on regular metra dataset before combining



In [20]:
obs_df.TIME_ON.value_counts()
# metra_df.Time.value_counts()

7:00 - 8:00 am         3526
4:00 - 5:00 pm         3290
5:00 - 6:00 pm         2827
8:00 - 9:00 am         2688
3:00 - 4:00 pm         2527
10:00 - 11:00 am       2053
9:00 - 10:00 am        2050
6:00 - 7:00 am         1905
7:00 - 8:00 pm         1801
11:00 am - 12:00 pm    1422
12:00 pm - 1:00 pm     1189
6:30 - 7:00 pm         1088
2:00 - 3:00 pm         1068
6:00 - 6:30 pm         1031
1:00 - 2:00 pm          705
8:00 - 9:00 pm          653
11:00 - 12:00 pm        525
Before 6:00 am          117
After 9:00 pm            26
Name: TIME_ON, dtype: int64

### Tour Purpose

* Work (home-based-work or work-based tour)
* School
* Other - use a hierarchy


In [21]:
purpose_ct = pd.crosstab(
    obs_df['ORIG_PURPOSE'],
    obs_df['DEST_PURPOSE'],
    values=obs_df['unlinked_weight_scaled'],
    aggfunc=sum,
    normalize='all',
    margins=True)*100
# purpose_ct = add_percentage_to_crosstab(purpose_ct)
# purpose_ct.fillna(0, inplace=True)
purpose_ct.applymap(lambda x: round(x,2))

DEST_PURPOSE,Discretionary,Home,K-12 school,Maintenance,University,Work,All
ORIG_PURPOSE,,,,,,,
Discretionary,2.00,5.48,0.12,0.83,0.43,1.00,9.86
Home,9.13,0.02,1.97,5.48,6.79,26.14,49.52
K-12 school,0.20,1.41,0.02,0.03,0.05,0.13,1.84
Maintenance,0.85,4.81,0.03,0.57,0.08,0.36,6.71
University,0.64,4.58,0.02,0.15,1.27,0.42,7.06
Work,1.93,21.39,0.02,0.69,0.37,0.62,25.02
All,14.74,37.68,2.18,7.75,8.99,28.67,100.00


In [22]:
def determine_tour_purpose(orig_purpose, dest_purpose):
    if (orig_purpose == 'Work') & (dest_purpose == 'Work'):
        return 'Work sub-tour'
    if (orig_purpose == 'Work') | (dest_purpose == 'Work'):
        return 'Work'
    if (orig_purpose == 'Other work') | (dest_purpose == 'Other work'):
        return 'Maintenance'
    if (orig_purpose == 'University') | (dest_purpose == 'University'):
        return 'University'
    if (orig_purpose == 'K-12 school') | (dest_purpose == 'K-12 school'):
        return 'School'
    if (orig_purpose == 'Maintenance') | (dest_purpose == 'Maintenance'):
        return 'Maintenance'
    if (orig_purpose == 'Discretionary') | (dest_purpose == 'Discretionary'):
        return 'Discretionary'
    if (orig_purpose == 'Home') & (dest_purpose != 'Home'):
        return dest_purpose
    if (orig_purpose != 'Home') & (dest_purpose == 'Home'):
        return orig_purpose
    if (orig_purpose == 'Home') & (dest_purpose == 'Home'):
        return 'Discretionary'
    if orig_purpose == dest_purpose:
        return orig_purpose
    return dest_purpose

obs_df['TOUR_PURPOSE'] = obs_df.apply(lambda row: determine_tour_purpose(row['ORIG_PURPOSE'], row['DEST_PURPOSE']), axis=1)

# home-home as other discrectionary
# Social/visiting vs. other discretionary
# Recreation & sporting in "other discretionary"


In [23]:
# use trip weights instead
obs_df['TOUR_PURPOSE'].value_counts(dropna=False)
obs_df.groupby(['TOUR_PURPOSE'])['linked_weight_scaled'].agg(sum).sort_index(level='TOUR_PURPOSE')


TOUR_PURPOSE
Discretionary     43847.228655
Maintenance       33937.855884
School             8937.711657
University        40840.006454
Work             143004.575059
Work sub-tour      1795.336742
Name: linked_weight_scaled, dtype: float64

In [24]:
obs_df.groupby(['TOUR_PURPOSE'])['unlinked_weight_scaled'].agg(sum).sort_index(level='TOUR_PURPOSE')

TOUR_PURPOSE
Discretionary     58242.161940
Maintenance       43958.250436
School            13257.389708
University        49020.267665
Work             183837.778131
Work sub-tour      2158.280051
Name: unlinked_weight_scaled, dtype: float64

In [25]:
# access/egress individually coded earlier
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# code access based on both access/egress
def determine_tour_mode(row):
    if (row['access_mode'] == 'PNR') | (row['egress_mode'] == 'PNR'):
        return 'PNR'
    if (row['access_mode'] == 'TNR') | (row['egress_mode'] == 'TNR'):
        return 'TNR'
    if (row['access_mode'] == 'KNR') | (row['egress_mode'] == 'KNR'):
        return 'KNR'
    return 'Walk'

obs_df['TOUR_MODE'] = obs_df.apply(lambda row: determine_tour_mode(row), axis=1)
access_df = obs_df.groupby(['TOUR_MODE'])['unlinked_weight_scaled'].agg(sum).sort_index(level='TOUR_MODE').to_frame()
access_df['Percent'] = access_df['unlinked_weight_scaled'] / access_df['unlinked_weight_scaled'].sum() * 100



print(access_df)

obs_df.groupby(['TOUR_MODE'])['unlinked_weight_scaled'].agg(sum).sort_index(level='TOUR_MODE')

           unlinked_weight_scaled  Percent
TOUR_MODE                                 
KNR                     19110.257    5.453
PNR                     39250.223   11.199
Walk                   292113.648   83.348


TOUR_MODE
KNR     19110.257
PNR     39250.223
Walk   292113.648
Name: unlinked_weight_scaled, dtype: float64

In [26]:
# derive TOD/use CMAP utilities for now

obs_df.loc[(obs_df['TIME_ON_CODE']>=5) & (obs_df['TIME_ON_CODE'] <=11), 'TOD'] = '9:01 am - 3:00 pm'
obs_df.loc[(obs_df['TIME_ON_CODE']>=6) & (obs_df['TIME_ON_CODE'] <=9), 'TOD'] = '10:01 am - 2:00 pm'


In [27]:
# Impute work tour purpose
# Limit to workers, tour purpose is maintenance/discretionary
# limit to weekdays? need to ask -- Day_Type_2 == 1

obs_df['TOUR_PURPOSE_INIT'] = obs_df['TOUR_PURPOSE']
obs_df['imputed_tour_purpose'] = ''

# calculate utility work tour based on model coefficients
obs_df['uWTP'] = -0.480

# original tour purpose
obs_df.loc[(obs_df['TOUR_PURPOSE_INIT'] == 'Discretionary'), 'uWTP'] = obs_df[obs_df['TOUR_PURPOSE_INIT'] == 'Discretionary']['uWTP'] + (-0.961)


# surveyed hour
obs_df.loc[(obs_df['TOD'] == '9:01 am - 3:00 pm'), 'uWTP'] = obs_df[(obs_df['TOD'] == '9:01 am - 3:00 pm')]['uWTP'] + (-1.21)
obs_df.loc[(obs_df['TIME_ON_CODE'] == 12), 'uWTP'] = obs_df[(obs_df['TIME_ON_CODE'] == 12)]['uWTP'] + (-0.19 ) # 4pm
obs_df.loc[(obs_df['TIME_ON_CODE'] == 13), 'uWTP'] = obs_df[(obs_df['TIME_ON_CODE'] == 13)]['uWTP'] + (-0.365) # 5pm
obs_df.loc[(obs_df['TIME_ON_CODE'] == 14) | (obs_df['TIME_ON_CODE'] == 15), 'uWTP'] = obs_df[(obs_df['TIME_ON_CODE'] == 14) | (obs_df['TIME_ON_CODE'] == 15)]['uWTP'] + (-0.932) # 6pm
obs_df.loc[(obs_df['TIME_ON_CODE'] == 16), 'uWTP'] = obs_df[(obs_df['TIME_ON_CODE'] == 16)]['uWTP'] + (-1.25) # 7pm
obs_df.loc[(obs_df['TIME_ON_CODE'] == 17), 'uWTP'] = obs_df[(obs_df['TIME_ON_CODE'] == 17)]['uWTP'] + (-1.46) # 8pm
obs_df.loc[(obs_df['TIME_ON_CODE'] == 18), 'uWTP'] = obs_df[(obs_df['TIME_ON_CODE'] == 18)]['uWTP'] + (-1.12) # >9pm

# access mode
obs_df.loc[(obs_df['TOUR_MODE'] == 'PNR'), 'uWTP'] = obs_df[(obs_df['TOUR_MODE'] == 'PNR')]['uWTP'] + (1.12) 
obs_df.loc[(obs_df['TOUR_MODE'] == 'KNR'), 'uWTP'] = obs_df[(obs_df['TOUR_MODE'] == 'KNR')]['uWTP'] + (1.78) 

# calculate probability
obs_df['expuWTP'] = np.exp(obs_df['uWTP'])
obs_df['probWTP'] = obs_df['expuWTP']/(1 + obs_df['expuWTP'])

print(obs_df['probWTP'].value_counts(dropna=False))

# Generate random number
np.random.seed(30)
obs_df['randn'] = np.random.uniform(0, 1, obs_df.shape[0])

obs_df.loc[(obs_df['randn'] < obs_df['probWTP']), 'imputed_tour_purpose'] = 'Work'
obs_df.loc[(obs_df['randn'] >= obs_df['probWTP']), 'imputed_tour_purpose'] = 'replace'


print(np.mean(obs_df['probWTP']))
print(obs_df[obs_df['imputed_tour_purpose'] == 'Work'].shape[0]/
      (obs_df[obs_df['imputed_tour_purpose'] == 'Work'].shape[0] + 
       obs_df[obs_df['imputed_tour_purpose'] == 'replace'].shape[0]))

obs_df.loc[(obs_df['hh_employed'] > 0) & 
           (obs_df['imputed_tour_purpose'] == 'Work') & 
           (obs_df['TOUR_PURPOSE_INIT'].isin(['Maintenance', 'Discretionary'])), 'TOUR_PURPOSE'] = obs_df[(obs_df['hh_employed'] > 0) & 
               (obs_df['imputed_tour_purpose'] == 'Work') & 
               (obs_df['TOUR_PURPOSE_INIT'].isin(['Maintenance', 'Discretionary']))]['imputed_tour_purpose']

obs_df.groupby(['TOUR_PURPOSE'])['linked_weight_scaled'].agg(sum).sort_index(level='TOUR_PURPOSE')



0.382    9221
0.156    3716
0.655    2331
0.338    1931
0.300    1659
0.191    1578
0.196    1314
0.151    1121
0.066     801
0.786     799
0.611     784
0.361     719
0.568     507
0.085     492
0.064     428
0.141     401
0.126     392
0.164     370
0.522     300
0.052     188
0.718     175
0.752     160
0.584     138
0.428     134
0.591     110
0.512     100
0.352      86
0.420      65
0.295      60
0.178      49
0.287      44
0.335      43
0.494      42
0.356      38
0.222      31
0.537      30
0.246      25
0.460      24
0.172      22
0.306      21
0.375      15
0.168      12
0.072       9
0.144       3
0.545       3
Name: probWTP, dtype: int64
0.33865062576825417
0.336787904627595


TOUR_PURPOSE
Discretionary    38829.714
Maintenance      27587.142
School            8937.712
University       40840.006
Work            154372.803
Work sub-tour     1795.337
Name: linked_weight_scaled, dtype: float64

In [28]:
# Derive observed/surveyed hour (as a sub for departure hour)


# # pd.to_datetime(obs_df['StartTime']).head()
# obs_df['surveyed_hour'] = pd.to_datetime(obs_df['StartTime']).dt.hour

# obs_df['surveyed_hour'].value_counts(dropna=False).sort_index(level='surveyed_hour')


In [29]:
# Impute work sub-tour purpose
# Limit to origin/destination not home

# calculate utility work subtour based on model coefficients
obs_df['uWST'] = -5.16

obs_df.loc[(obs_df['ORIG_PURPOSE'] == 'Work') | (obs_df['DEST_PURPOSE'] == 'Work'), 'uWST'] = \
    obs_df[(obs_df['ORIG_PURPOSE'] == 'Work') | (obs_df['DEST_PURPOSE'] == 'Work')]['uWST'] + (2.68 )
obs_df.loc[(obs_df['TOD'] == '10:01 am - 2:00 pm'), 'uWST'] = obs_df[(obs_df['TOD'] == '10:01 am - 2:00 pm')]['uWST'] + (1.63)
obs_df.loc[(obs_df['TIME_ON_CODE'] == 8), 'uWST'] = obs_df[(obs_df['TIME_ON_CODE'] == 8)]['uWST'] + (0.458)



# calculate probability
obs_df['expuWST'] = np.exp(obs_df['uWST'])
obs_df['probWST'] = obs_df['expuWST']/(1 + obs_df['expuWST'])

print(obs_df['probWST'].value_counts(dropna=False))

# Generate random number
obs_df['randn'] = np.random.uniform(0, 1, obs_df.shape[0])

obs_df.loc[(obs_df['randn'] < obs_df['probWST']), 'imputed_tour_purpose'] = 'Work sub-tour'
obs_df.loc[(obs_df['randn'] >= obs_df['probWST']), 'imputed_tour_purpose'] = 'replace'


print(np.mean(obs_df['probWST']))
print(obs_df[obs_df['imputed_tour_purpose'] == 'Work sub-tour'].shape[0]/
      (obs_df[obs_df['imputed_tour_purpose'] == 'Work sub-tour'].shape[0] + 
       obs_df[obs_df['imputed_tour_purpose'] == 'replace'].shape[0]))

obs_df.loc[(obs_df['imputed_tour_purpose'] == 'Work sub-tour') & 
           (obs_df['ORIG_PURPOSE'] != 'Home') & 
           (obs_df['DEST_PURPOSE'] != 'Home'), 'TOUR_PURPOSE'] = obs_df[(obs_df['imputed_tour_purpose'] == 'Work sub-tour') & 
           (obs_df['ORIG_PURPOSE'] != 'Home') & 
           (obs_df['DEST_PURPOSE'] != 'Home')]['imputed_tour_purpose']

obs_df.groupby(['TOUR_PURPOSE'])['linked_weight_scaled'].agg(sum).sort_index(level='TOUR_PURPOSE')



0.077    15160
0.006     9437
0.028     2968
0.299     1737
0.044      754
0.403      435
Name: probWST, dtype: int64
0.06686333099126178
0.06962710307959726


TOUR_PURPOSE
Discretionary    38686.081
Maintenance      27508.104
School            8935.653
University       40686.600
Work            152001.199
Work sub-tour     4545.077
Name: linked_weight_scaled, dtype: float64

In [30]:

obs_df.COUNT_MEMBER_HH_CODE.value_counts()


2.000     8502
1.000     6140
3.000     5457
4.000     4733
5.000     2417
6.000     1010
7.000      321
10.000     231
8.000      197
9.000       77
Name: COUNT_MEMBER_HH_CODE, dtype: int64

In [31]:
# Impute joint tour
# Limit to hhsize >1 and maintenance/discretionary tour purpose (after prev steps)

# calculate utility fully joint tour based on model coefficients
obs_df['uFJT'] = -2.65

# surveyed hour
obs_df.loc[(obs_df['TOD'] == '9:01 am - 3:00 pm'), 'uFJT'] = obs_df[(obs_df['TOD'] == '9:01 am - 3:00 pm')]['uFJT'] + (0.567)
obs_df.loc[(obs_df['TIME_ON_CODE'] == 12), 'uWTP'] = obs_df[(obs_df['TIME_ON_CODE'] == 12)]['uWTP'] + (0.3547 ) # 4pm
obs_df.loc[(obs_df['TIME_ON_CODE'] == 13), 'uWTP'] = obs_df[(obs_df['TIME_ON_CODE'] == 13)]['uWTP'] + (0.643) # 5pm
obs_df.loc[(obs_df['TIME_ON_CODE'] == 14) | (obs_df['TIME_ON_CODE'] == 15), 'uWTP'] = \
    obs_df[[(obs_df['TIME_ON_CODE'] == 14) | (obs_df['TIME_ON_CODE'] == 15)]['uWTP'] + (1.02 ) # 6pm
obs_df.loc[(obs_df['TIME_ON_CODE'] == 15), 'uWTP'] = obs_df[(obs_df['TIME_ON_CODE'] == 16)]['uWTP'] + (1.23) # 7pm
obs_df.loc[(obs_df['TIME_ON_CODE'] == 16), 'uWTP'] = obs_df[(obs_df['TIME_ON_CODE'] == 17)]['uWTP'] + ( 0.997  ) # 8pm
obs_df.loc[(obs_df['TOUR_PURPOSE_INIT'] == 'Discretionary'), 'uFJT'] = obs_df[(obs_df['TOUR_PURPOSE_INIT'] == 'Discretionary')]['uFJT'] + (0.227)
obs_df.loc[(obs_df['TOUR_PURPOSE_INIT'] == 'Maintenance'), 'uFJT'] = obs_df[(obs_df['TOUR_PURPOSE_INIT'] == 'Maintenance')]['uFJT'] + (-0.197)

# calculate probability
obs_df['expuFJT'] = np.exp(obs_df['uFJT'])
obs_df['probFJT'] = obs_df['expuFJT']/(1 + obs_df['expuFJT'])

print(obs_df['probFJT'].value_counts(dropna=False))

# Generate random number
obs_df['randn'] = np.random.uniform(0, 1, obs_df.shape[0])

obs_df.loc[(obs_df['randn'] < obs_df['probFJT']), 'joint_tour_type'] = 'Joint'
obs_df.loc[(obs_df['randn'] >= obs_df['probFJT']), 'joint_tour_type'] = 'Ind'


print(np.mean(obs_df['probFJT']))
print(obs_df[obs_df['joint_tour_type'] == 'Joint'].shape[0]/
      (obs_df[obs_df['joint_tour_type'] == 'Joint'].shape[0] + 
       obs_df[obs_df['joint_tour_type'] == 'Ind'].shape[0]))

obs_df.loc[(obs_df['COUNT_MEMBER_HH_CODE'] > 1) & # hhsize > 1
           (obs_df['TOUR_PURPOSE'] == 'Discretionary') & 
           (obs_df['joint_tour_type'] == 'Joint'), 'TOUR_PURPOSE'] = 'Joint-Discretionary'

obs_df.loc[(obs_df['TOUR_PURPOSE'] == 'Discretionary'), 'TOUR_PURPOSE'] = 'Ind-Discretionary'

obs_df.loc[(obs_df['COUNT_MEMBER_HH_CODE'] > 1) & # hhsize > 1
           (obs_df['TOUR_PURPOSE'] == 'Maintenance') & 
           (obs_df['joint_tour_type'] == 'Joint'), 'TOUR_PURPOSE'] = 'Joint-Maintenance'

obs_df.loc[(obs_df['TOUR_PURPOSE'] == 'Maintenance'), 'TOUR_PURPOSE'] = 'Ind-Maintenance'


obs_df.groupby(['TOUR_PURPOSE'])['linked_weight_scaled'].agg(sum).sort_index(level='TOUR_PURPOSE')



SyntaxError: invalid syntax (<ipython-input-31-610a16155018>, line 13)

In [ ]:
obs_df.groupby(['TOUR_PURPOSE'])['linked_weight_scaled'].agg(sum).sort_index(level='TOUR_PURPOSE')


## Combine with reverse metra data

In [ ]:
make_percentage_bar_plot(
    variable='auto_suff',
    title="Auto Sufficiency",
    xlabel="Auto Sufficiency",
    xlabel_rotation=0)



In [ ]:
make_percentage_bar_plot(
    variable='TOUR_PURPOSE',
    title="Inferred Tour Purpose",
    xlabel="Tour Purpose",
    xlabel_rotation=90)


In [ ]:
obs_df['TOUR_MODE'] = obs_df.apply(lambda row: determine_tour_mode(row), axis=1)
access_df = obs_df.groupby(['TOUR_MODE'])['linked_weight_scaled'].agg(sum).sort_index(level='TOUR_MODE').to_frame()
access_df['Percent'] = access_df['linked_weight_scaled'] / access_df['linked_weight_scaled'].sum() * 100



print(access_df)

obs_df.groupby(['TOUR_MODE'])['linked_weight_scaled'].agg(sum).sort_index(level='TOUR_MODE')

## Summary tables

### Boardings by line

In [ ]:

obs_df.ROUTE.value_counts()
obs_df.groupby(['ROUTE'])['unlinked_weight_scaled'].agg(sum).sort_index(level='ROUTE')

In [ ]:
mode_df = obs_df.groupby(['TOUR_MODE'])['linked_weight_scaled'].agg(sum).sort_index(level='TOUR_MODE').to_frame()
mode_df['Percent'] = mode_df['linked_weight_scaled'] / mode_df['linked_weight_scaled'].sum() * 100
display(HTML(mode_df.to_html()))

### Linked trips by transit mode + tour purpose

In [ ]:
mode_purpose_ct = pd.crosstab(obs_df['TOUR_PURPOSE'], obs_df['TOUR_MODE'], values=obs_df['linked_weight_scaled'], aggfunc=sum, margins=True)
mode_purpose_ct = add_percentage_to_crosstab(mode_purpose_ct)
mode_purpose_ct.fillna(0, inplace=True)
mode_purpose_ct.applymap(lambda x: round(x,2))


### Linked trips by tour purpose, auto sufficiency, and access mode

In [ ]:
purpose_suff_access_ct = pd.crosstab(obs_df['TOUR_PURPOSE'], [obs_df['auto_suff'], obs_df['access_mode']], values=obs_df['linked_weight_scaled'], aggfunc=sum, margins=True)
purpose_suff_access_ct = add_percentage_to_crosstab(purpose_suff_access_ct)
purpose_suff_access_ct.fillna(0, inplace=True)
purpose_suff_access_ct.applymap(lambda x: round(x,2))

## Write coded output

In [ ]:
obs_df.to_csv(os.path.join(output_path, 'processed_obs.csv'))